# Working through the "Atacama Soil Microbiome" Tutorial in Qiime2
- The original tutorial is here in the official Qiime2 docs: https://docs.qiime2.org/2021.2/tutorials/atacama-soils/
- This workshop was also adapted from a previous workshop hosted by 
Joslynn Lee: https://cyverse-jupyter-qiime2.readthedocs-hosted.com/en/latest/

### Learning objectives:
- Understand the analysis steps and core concepts of the Qiime2 software
- Generate plots to visualize biodiversity results from metabarcoding data

### How this notebook works:
- Code is fill-in-the-blank, follow along with us as we work through it together!
- Stuck or Lost? We have many points of redemption throughout; just download the output files and visualize them here: https://view.qiime2.org/

### Steps covered:
0. Setting up the Jupyter notebook and input files
1. Demultiplexing paired-end sequences
2. Denoising using DADA2
3. Generate a tree for phylogenetic diversity analyses
4. Alpha and beta diversity analysis
5. Bonus: Taxonomic assignment

# 0. Setting up the Jupyter and input files

This first step is only because we are using a Jupyter notebook for this workshop. You will not need to run these few lines of code when working with your own datasets on your computer. 

In [ ]:
pip3 install git+https://github.com/regebro/tzlocal
qiime dev refresh-cache

Generally on your local computer, you will need to activate Qiime2 using `conda`. Here, we have pre-loaded it, so you will not need to run this command: 

In [ ]:
# conda activate qiime2

Now, we will make a directory (folder) for our tutorial, as well as a directory for our raw, paired-end sequences. 
`mkdir` = make a directory

In [ ]:
mkdir qiime2-atacama-tutorial
cd qiime2-atacama-tutorial

Then, we are going to download the sample metadata file from the Qiime2 website. 

In [ ]:
wget \
  -O "sample-metadata.tsv" \
  "https://data.qiime2.org/2021.2/tutorials/atacama-soils/sample_metadata.tsv"

We are going to put these raw sequences in their own directory. You will download three `fastq.gz` files, corresponding to the forward, reverse, and barcode (i.e., index) reads. These files contain a subset of the reads in the full data set generated for this study, which allows for the following commands to be run relatively quickly, however, we will perform additional subsampling in this tutorial to further improve the run time.

In [ ]:
mkdir emp-paired-end-sequences

wget \
  -O "emp-paired-end-sequences/forward.fastq.gz" \
  "https://data.qiime2.org/2021.2/tutorials/atacama-soils/10p/forward.fastq.gz"

wget \
  -O "emp-paired-end-sequences/reverse.fastq.gz" \
  "https://data.qiime2.org/2021.2/tutorials/atacama-soils/10p/reverse.fastq.gz"

wget \
  -O "emp-paired-end-sequences/barcodes.fastq.gz" \
  "https://data.qiime2.org/2021.2/tutorials/atacama-soils/10p/barcodes.fastq.gz"

# 1. Demultiplexing paired-end sequences

To analyze these data, the sequences that you just downloaded must first be imported into an artifact of type `EMPPairedEndSequences`. We will not be going into the details of importing into Qiime2 to create Qiime2 artifacts, but you can find more information on that here: https://docs.qiime2.org/2021.4/tutorials/importing/

In [ ]:
qiime tools import \
   --type EMPPairedEndSequences \
   --input-path emp-paired-end-sequences \
   --output-path emp-paired-end-sequences.qza

We will now demultiplex the sequence reads. This requires the sample metadata file, specifically the column in the file that contains the per-sample barcodes. Here, the column is called `barcode-sequence`. Because the barcode reads are the reverse complement of those included in the sample metadata file, we can include the `--p-rev-comp-mapping-barcodes` parameter. One output of this file is looking at a summary of how many sequences were obtained per sample. 

In the `--m-barcodes-column` where there is `[COLUMN NAME]`, type in `barcode-sequence` to specify the column that the barcodes are in from the metadata file. This line should look like `--m-barcodes-column barcode-sequence \`

The below code may take a few minutes to run.

In [ ]:
qiime demux emp-paired \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-column [COLUMN NAME] \
  --p-rev-comp-mapping-barcodes \
  --i-seqs emp-paired-end-sequences.qza \
  --o-per-sample-sequences demux-full.qza \
  --o-error-correction-details demux-details.qza

Let's subsample the data - this will speed up the tutorial and run time, and demonstrate the functionality. Other uses of subsampling need to be thought through with reasonable justification. 

Here, where you see `[COMMAND]`, complete the qiime command with `summarize` to take a look at summary information of our data after subsampling. The full line should be `qiime demux summarize` 

In [ ]:
qiime demux subsample-paired \
  --i-sequences demux-full.qza \
  --p-fraction 0.3 \
  --o-subsampled-sequences demux-subsample.qza

qiime demux [COMMAND] \
  --i-data demux-subsample.qza \
  --o-visualization demux-subsample.qzv

We can view the summary of our subsampled dataset to examine how many sequence counts were in each sample. Here, there are 75 samples in the data and the last 20 or so of the rows in the table have fewer than 100 reads in them, which can be filtered out of the data:

In [ ]:
qiime tools export \
  --input-path demux-subsample.qzv \
  --output-path ./demux-subsample/

qiime demux filter-samples \
  --i-demux demux-subsample.qza \
  --m-metadata-file ./demux-subsample/per-sample-fastq-counts.tsv \
  --p-where 'CAST([forward sequence count] AS INT) > 100' \
  --o-filtered-demux demux.qza

# 2. Denoising using DADA2
We will look at the sequence quality based on 10,000 randomly selected reads from the subsampled and filtered data, then denoise it based on the quality plots. There will be two plots showing quality scores - one for the forward reads and one for the reverse reads. These plots will help us determine what trimming parameters we want to denoise with, using DADA2. On local uses of Qiime2, you can view outputs using the qiime tools view command, followed by your .qzv file. Here, we will view the quality plots by downloading `demux-subsample.qzv` on https://view.qiime2.org/

In this example, we have 150-base forward and reverse reads. We need to keep in mind that the paired reads need to be long enough to overlap so they (the forward and reverse sequences) can merge. We will trim off the first 13 bases of the forward and reverse reads, but will not trim the end of the sequences, to avoid reducing the read length too much. You do not have to keep the trimming or truncation lengths the same for your forward and reverse sequences, but we are doing that here:

Fill in the `p-trim-left-f` and `--p-trim-left-r` parameters with `13` where there is a `[NUMBER]` prompt. 

The following code may take a few minutes to run. 

In [ ]:
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left-f [NUMBER] \
  --p-trim-left-r [NUMBER] \
  --p-trunc-len-f 150 \
  --p-trunc-len-r 150 \
  --o-table table.qza \
  --o-representative-sequences rep-seqs.qza \
  --o-denoising-stats denoising-stats.qza

Now, we have artifacts containing the feature table, corresponding feature sequences, and DADA2 denoising statistics. We can generate summaries of these data:

Fill in the `[DATA FILE]` blank after `--i-data` with `rep-seqs.qza`, which is the output file from the denoising step from above. In addition, we can take a look at `denoising-stats.qza` by putting it in where `[DENOISING METADATA]` is after `--m-input-file`.

In [ ]:
qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

qiime feature-table tabulate-seqs \
  --i-data [DATA FILE] \
  --o-visualization rep-seqs.qzv

qiime metadata tabulate \
  --m-input-file [DENOISING METADATA] \
  --o-visualization denoising-stats.qzv

Let's view some of the `.qzv` files on https://view.qiime2.org/ to look at how many sequences remain in each sample.

## 3. Generate a tree for phylogenetic diversity analyses
In addition to counts of features per sample (i.e., the data in the `FeatureTable[Frequency]` Qiime2 artifact), some biodiversity metrics require a rooted phylogeentic tree relating the features to one another. These phylogenetic diveresity metrics include Faith's Phylogenetic Diversity and weighted and unweighted UniFrac. Phylogenetic trees in Qiime2 are stored as a `Phylogeny[Rooted]` Qiime2 artifact. We will use the `align-to-tree-mafft-fasttree` pipeline from the `q2-phylogeny` plugin. 

First, the pipeline uses the mafft program to perform a multiple sequence alignment of the sequences in our `FeatureData[Sequence]` to create a `FeatureData[AlignedSequence]` QIIME 2 artifact. Next, the pipeline masks (or filters) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree. Following that, the pipeline applies FastTree to generate a phylogenetic tree from the masked alignment. The FastTree program creates an unrooted tree, so in the final step in this section, midpoint rooting is applied to place the root of the tree at the midpoint of the longest tip-to-tip distance in the unrooted tree.

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

## 4. Alpha and beta diversity analysis
QIIME2 offers diversity analyses through the `q2-diversity` plugin, which supports computing alpha and beta diversity metrics, applying related statistical tests, and generating interactive visualizations. We will first use the `core-metrics-phylogenetic method`, which rarefies a `FeatureTable[Frequency]` to a user-specified depth,  computes several alpha and beta diversity metrics, and generates principle coordinates analysis (PCoA) plots using Emperor for each of the beta diversity metrics. The metrics computed by default are:

Alpha diversity

   - Shannon’s diversity index (a quantitative measure of community richness)
   - Observed Features (a qualitative measure of community richness)
   - Faith’s Phylogenetic Diversity (a qualitiative measure of community richness that incorporates phylogenetic relationships between the features)
   - Evenness (or Pielou’s Evenness; a measure of community evenness)

Beta diversity

   - Jaccard distance (a qualitative measure of community dissimilarity)
   - Bray-Curtis distance (a quantitative measure of community dissimilarity)
   - unweighted UniFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)
   - weighted UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)

An important parameter that needs to be provided to this script is `--p-sampling-depth`, which is the even sampling (i.e. rarefaction) depth. Because most diversity metrics are sensitive to different sampling depths across different samples, this script will randomly subsample the counts from each sample to the value provided for this parameter. For example, if you provide `--p-sampling-depth 500`, this step will subsample the counts in each sample without replacement so that each sample in the resulting table has a total count of 500. If the total count for any sample(s) are smaller than this value, those samples will be dropped from the diversity analysis. Choosing this value is tricky. We recommend making your choice by reviewing the information presented in the table.qzv file that was created above. Choose a value that is as high as possible (so you retain more sequences per sample) while excluding as few samples as possible.

Looking at the `table.qzv` Qiime2 artifact we just generated, in particular the `'Interative Sample Detail'` tab in that visualization, what value would you choose to pass for `--p-sampling-depth`? How many samples will be excluded from your analysis based on this choice? How many total sequences will you be analyzing in the core-metrics-phylogenetic command?

We are also going to filter out a site that had missing metadata measurements for convenience of downstream beta diversity analyses. We will then use `table-filtered.qza` for the remaining analyses.

Fill in the `[NUMBER]` prompt after `--p-sampling-depth` with `500`. 

In [ ]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-where "'site-name' = 'BAQ4697'" \
  --p-exclude-ids True \
  --o-filtered-table table-filtered.qza

qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table-filtered.qza \
  --p-sampling-depth [NUMBER] \
  --m-metadata-file sample-metadata.tsv \
  --output-dir core-metrics-results

We set the `--p-sampling-depth` parameter to 500. This value was chosen based on the `table.qzv` output: it allowed for most of the samples to be retained while also dropping samples that had a low number of sequences. Losing a disproportionate number of samples from one metadata category is not ideal. You want to strike a balance between total seqeuences analyzed and number of samples retained. 

On an Illumina run, it is common to see a few samples with very low sequence counts. These should typically be removed from analyses by setting a larger value for the sampling depth at this stage.

After computing diversity metrics, we can begin to explore the microbiome community composition of samples in the context of sample metadata. This information is present in the sample metadata file that was downloaded earlier.

We'll first test for association between categorical metadata columns and alpha diversity data. We'll do that here for Shannon's alpha diversity and evenness metrics. 

In [ ]:
qiime diversity \
  --i-alpha-diversity core-metrics-results/shannon_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/shannon-group-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

 Next, we will analyze the sample composition in the contet of categorical metadata using PERMANOVA (Anderson, 2001) using the `beta-group-significance` command. The commands will test whether distances between samples within a group are more similar to each other than they are to samples from the other groups. PERMANOVA can also perform pairwise tests that allow you to determine which specific pairs of groups differ from one another. We will do this here with `--p-pairwise`. This command can be slow to run since it's based on permutation test. We will run `beta-group-significance` on specific metadata columns to reduce computation time. Here, will apply this to our unweighted UniFrac distances, using two sample metadata columns, as follows:
 
Fill in the last part of the code where you see `[FINAL PARAMETER]` with `--p-pairwise` so we can perform pairwise comparisons. 

In [ ]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column vegetation \
  --o-visualization core-metrics-results/unweighted-unifrac-vegetation-significance.qzv \
  [FINAL PARAMETER]

## 5. Bonus: Taxonomic Assignment
You can consult taxonomic assignment steps here: https://docs.qiime2.org/2021.4/tutorials/feature-classifier/. Unfortunatley, this Jupyter notebook may not have the computational power to run the taxonomic assignment step `classify-consensus-vsearch`, but the general steps are outlined below. Also, the more widely used algorithm is now `classify-sklearn` (this one throws an error on Jupyter notebook, but works great in other use cases).  

### Essential steps:
0. Obtain and import reference reference datasets
1. Extract reference reads
2. Train the classifier
3. Use the classifier

* Note that QIIME2 has pre-trained classifiers for some well-known databases available, but taxonomic assignment performs better when you train the classifier to your primers/data. Trimming and training taxonomic data takes a while, so below is code for a pre-trained classifier for our primers of interest. Also, training classifiers is not a prerequisite to using QIIME2's taxonomic assignment tools. 

#### NOTE: Unfortunately, we don't have the computational power to execute the below code to completion; it's just here for your reference.

### Make directory for taxonomic assignment

In [ ]:
mkdir training-feature-classifiers
cd training-feature-classifiers

### Get pre-trained SILVA database, since this dataset uses the v4 region of the 16s rRNA gene with the 515F/806R primers

In [ ]:
wget \
  -O "silva-otus.qza" \
"https://data.qiime2.org/2021.4/common/silva-138-99-seqs-515-806.qza"

wget \
 -O "silva-taxa.qza" \
"https://data.qiime2.org/2021.4/common/silva-138-99-tax-515-806.qza"

### Use the classifier

In [ ]:
qiime feature-classifier classify-consensus-vsearch \
    --i-reference-reads silva-otus.qza \
    --i-reference-taxonomy silva-taxa.qza \
    --i-query ../rep-seqs.qza \
    --p-perc-identity 0.97 \
    --o-classification vsearch-taxonomy.qza \
    --verbose

### View results

In [ ]:
qiime metadata tabulate \
  --m-input-file vsearch-taxonomy.qza \
  --o-visualization vsearch-taxonomy.qzv

# Questions? 